**Resources**
<!-- {{< youtube akZ8JJ4gGLs >}} -->
<!-- {{< tweet 1266579585417613312 >}} -->
[Smoothstep](https://smoothstep.io/)

- Breadth First Search
- Dijkstra's algorithm/Fast Marching Method for solving the Eikonal equation?
- BFS is generalized as Disjkstra, which is generalized as Fast Marching, then as Ordered Upwind method, then as Anisotropic Fast Marching

- Hex mazes
- Bellman-Ford Algorithm
- Fast Marching Algorithm (FMM), eikonal equation
- Art: https://troika.uk.com/work/troika-labyrinth/
- https://en.wikipedia.org/wiki/Lee_algorithm
- [Procedural Content Generation: Mazes](http://pcg.wikidot.com/pcg-algorithm:maze)
- [Wikipedia Maze Generation Algorithms](https://en.wikipedia.org/wiki/Maze_generation_algorithm)


Solving a Maze
- https://en.wikipedia.org/wiki/Maze-solving_algorithm


Per wikipedia:
> The best-known rule for traversing mazes is the wall follower, also known as either the left-hand rule or the right-hand rule. If the maze is simply connected, that is, all its walls are connected together or to the maze's outer boundary, then by keeping one hand in contact with one wall of the maze the solver is guaranteed not to get lost and will reach a different exit if there is one; otherwise, the algorithm will return to the entrance having traversed every corridor next to that connected section of walls at least once. The algorithm is a depth-first in-order tree traversal.

For autonomous agents I perhaps one of these will be suitable.
- [Maze Routing Algorithm](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Maze-routing_algorithm)
- [Shortest Path Algorithms](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Shortest_path_algorithm)


What do I really want to do here?
- Ultimately, I want to get back to working on the game. 
For NPCs:
  - Find the shortest distance between to points. Example enter/exit the maze.
  - Follow another character in the maze.
  - Search for an item in the maze.
  - animate moving a maze explorer
  - draw out the solution and possibly rejected solutions.
  - The AI should only have knowledge of the rooms its visited and the room it's
    currently in. Is that true though?

Questions
- What algorithm does Godot use?
  A* and 

Next Steps
- Try doing a simple animation using iPyCanvas. 
- Need a way to make an animated gif from the animation if I'm going to be able
  to put it on the blog.
- Agent that just walks the maze randomly.
- Agent that uses an algorithm for traversing.
- Multiple agents interacting (chasing, hiding)
- It may make sense to generate the same maze every time. Perhaps seed the random number generator. 

In [ ]:
# Load python code.
%load_ext autoreload
%autoreload 2

In [1]:
# Generate a maze.
from ipycanvas import Canvas,  hold_canvas

from generation.structures import Point
from generation.maze import Maze
from generation.generators.random_backtracer import generate_maze_walls
from generation.renderers.wall_drawer import draw_maze, animate_drawing_by_rooms
from generation.renderers.agents_renderer import draw_agents
from generation.npc import Agent

maze: Maze = Maze(40, 40)
generate_maze_walls(maze)
canvas = Canvas(width=800, height=800)

# Create a new function animate_drawing_maze.
display(canvas)
draw_maze(maze, canvas)

# I want a couple of agents trying to escape the maze.
rogue = Agent('red')

# Place the rouge in the center of the maze
rogue.location = Point(maze.width/2, maze.height/2)

agents = []
agents.append(rogue)

draw_agents(agents, canvas)

Exit cell: Point(x=28, y=39)


Canvas(height=800, width=800)

In [ ]:
from ipywidgets import Output
from ipycanvas import Canvas, hold_canvas
from threading import Thread
import numpy as np
import time
from IPython.display import display, Image
from moviepy.editor import ImageSequenceClip

out = Output()

# 1000/125 = 8 FPS
SLEEP_TIME_SEC:float = 0.125

class Renderer:
    def __init__(self, canvas):
        self.frames = []
        self.canvas = canvas
        self.canvas.observe(self.get_array, 'image_data')
        self.setup_canvas()
        
    @out.capture()
    def get_array(self, *args, **kwargs):
        print("get_array called")
        arr = self.canvas.get_image_data()
        self.frames.append(arr)
        
    def setup_canvas(self):
        self.canvas.fill_style = 'white'
        self.canvas.fill_rect(0,0,self.canvas.width, self.canvas.height)
        
    def render(self, x):
        self.canvas.stroke_line(50, 10+(x*5), 150, 10+(x*5))
        
class MyThread(Thread):
    def __init__(self, renderer):
        super(MyThread, self).__init__()
        self.renderer = renderer
    
    def run(self):
        for i in range(8):
            self.renderer.render(i)
            time.sleep(SLEEP_TIME_SEC)
        self.on_complete()    

    def on_complete(self):
        self.save_gif()
        
    def save_gif(self):
        # save it as a gif
        print(f"Number of frames: {len(self.renderer.frames)}")
        clip = ImageSequenceClip(list(self.renderer.frames), fps=8)
        clip.write_gif('test.gif', fps=8)
        Image('test.gif')

# display(out)
# canvas = Canvas(width=200, height=200, sync_image_data=True)
# renderer = Renderer(canvas)
# thread = MyThread(renderer)

# display(canvas)
# thread.start()

In [ ]:
# The Main Entry Point

SAVE_ANIMATED_GIF = False

display(out)
canvas = Canvas(width=200, height=200, sync_image_data=True)
renderer = Renderer(canvas, SAVE_ANIMATED_GIF)
thread = RenderingThread(renderer, SAVE_ANIMATED_GIF)

display(canvas)
thread.start()